In [ ]:
import json
import math
import copy
import pandas as pd
from collections import defaultdict
def tree(): return defaultdict(tree)

In [ ]:
data = ''
with open('..\game_list.json', 'r') as file:
    for line in file:
        data += line
    if data != '':
        data = json.loads(data)
           
id_ = data[-1]['game_id']
data = ''

with open(f'{id_}/game_record.json', 'r') as file:
    for line in file:
        data += line        
rounds = json.loads(data)

In [ ]:
def get_name(bot, r):
    return rounds[r][bot]['name']

def get_method(bot, r):
    return rounds[r][bot]['used']

def get_points(bot, r):
    return rounds[r][bot]['points']

def get_time(bot, r):
    return rounds[r][bot]['time']

def get_round_num(r):
    return rounds[r]['round'].split('/')[0]

def get_winner(r):
    dictionary = ['DRAW', get_name('bot_1', 0), get_name('bot_2', 0)]
    return dictionary[rounds[r]['winner']]

def get_adv(r):
    dictionary = ['time', get_name('bot_1', 0), get_name('bot_2', 0)]
    return dictionary[rounds[r]['advantage']]

In [ ]:
methods = {'NOP()': 0, 'PATCH()': 3, 'SCAN()': 3, 'OVERLOAD()': 4, 'OVERHEAR()': 1, 'EXPLOIT()': 2, 'INFECT()': 4}
bot = tree()
general = tree()
name1, name2 = get_name('bot_1', 0), get_name('bot_2', 0)
bot['USED'][name1] = []
bot['USED'][name2] = []
bot['POINTS'][name1] = []
bot['POINTS'][name2] = []
general['RESULT'] = []
general['ADVANTAGE'] = []
for m in methods.keys():
    bot['WON_WITH'][name1][m] = 0
    bot['WON_WITH'][name2][m] = 0
    bot['EARN_BY'][name1][m] = 0
    bot['EARN_BY'][name2][m] = 0
    bot['LOST_WITH'][name1][m] = 0
    bot['LOST_WITH'][name2][m] = 0
    
for i in range(len(rounds)):
    bot['USED'][name1].append(get_method('bot_1', i))
    bot['USED'][name2].append(get_method('bot_2', i))
    bot['POINTS'][name1].append(get_points('bot_1', i))
    bot['POINTS'][name2].append(get_points('bot_2', i))
    general['RESULT'].append(get_winner(i))
    general['ADVANTAGE'].append(get_adv(i))
    
    if get_winner(i) == name1:
        bot['WON_WITH'][name1][get_method('bot_1', i)] += 1
        bot['EARN_BY'][name1][get_method('bot_1', i)] += methods[get_method('bot_1', i)]
        bot['LOST_WITH'][name2][get_method('bot_2', i)] += 1
    elif get_winner(i) == name2:
        bot['WON_WITH'][name2][get_method('bot_2', i)] += 1
        bot['EARN_BY'][name2][get_method('bot_2', i)] += methods[get_method('bot_2', i)]
        bot['LOST_WITH'][name1][get_method('bot_1', i)] += 1

In [ ]:
for m in methods.keys():
    bot['METHS_OVER_ROUNDS'][name1][m] = []
    bot['METHS_OVER_ROUNDS'][name2][m] = []
bot1_meths = {'NOP()': 0, 'PATCH()': 0, 'SCAN()': 0, 'OVERLOAD()': 0, 'OVERHEAR()': 0, 'EXPLOIT()': 0, 'INFECT()': 0}
bot2_meths = {'NOP()': 0, 'PATCH()': 0, 'SCAN()': 0, 'OVERLOAD()': 0, 'OVERHEAR()': 0, 'EXPLOIT()': 0, 'INFECT()': 0}

for i in range(len(rounds)):
    m1 = get_method('bot_1', i)
    m2 = get_method('bot_2', i)
    bot1_meths[m1] += 1
    bot2_meths[m2] += 1
    for m in methods.keys():
        bot['METHS_OVER_ROUNDS'][name1][m].append(copy.copy(bot1_meths[m]))
        bot['METHS_OVER_ROUNDS'][name2][m].append(copy.copy(bot2_meths[m]))

In [ ]:
dfwinner = pd.read_json(json.dumps(general))
dfwinner['RESULT'].value_counts().plot.pie(subplots=True, figsize=(10, 10), autopct='%.2f', title=["ROUNDS WON BY"])

In [ ]:
dfwinner['ADVANTAGE'].value_counts().plot.pie(subplots=True, figsize=(10, 10), autopct='%.2f', title=["ADVANTAGE AFTER ROUNDS"])

In [ ]:
df = pd.read_json(json.dumps(bot))

In [ ]:
s = pd.Series(df['METHS_OVER_ROUNDS'][name1])
s = pd.DataFrame({'NOP()': s['NOP()'], 'PATCH()': s['PATCH()'],
                  'SCAN()': s['SCAN()'], 'OVERLOAD()': s['OVERLOAD()'],
                 'OVERHEAR()':s['OVERHEAR()'], 'EXPLOIT()': s['EXPLOIT()'], 
                 'INFECT()': s['INFECT()']})
s.plot(figsize=(30, 10), grid=True, title=f"{name1} - METHODS USED OVER ROUNDS")

In [ ]:
s.plot.area(figsize=(30, 20), layout=(2,4), grid=True, subplots=True, title=f"{name1} - METHODS USED OVER ROUNDS")

In [ ]:
s = pd.Series(df['METHS_OVER_ROUNDS'][name2])
s = pd.DataFrame({'NOP()': s['NOP()'], 'PATCH()': s['PATCH()'],
                  'SCAN()': s['SCAN()'], 'OVERLOAD()': s['OVERLOAD()'],
                 'OVERHEAR()':s['OVERHEAR()'], 'EXPLOIT()': s['EXPLOIT()'], 
                 'INFECT()': s['INFECT()']})
s.plot(figsize=(30, 10), grid=True, title=f"{name2} - METHODS USED OVER ROUNDS")

In [ ]:
s.plot.area(figsize=(30, 20), layout=(2,4), grid=True, subplots=True, title=f"{name2} - METHODS USED OVER ROUNDS")

In [ ]:
df['POINTS'][name1] = pd.Series(df['POINTS'][name1])
df['POINTS'][name2] = pd.Series(df['POINTS'][name2])

In [ ]:
df_p = pd.DataFrame({name1: df['POINTS'][name1],
                    name2: df['POINTS'][name2]})
df_p.plot.area(figsize=(30, 10), grid=True, stacked=False, title="POINTS OVER ROUNDS")

In [ ]:
df_p[f'{name1} norm'] = df_p[name1] - df_p[name2]
df_p[f'{name2} norm'] = df_p[name2] - df_p[name1]
df_p.drop(name1, axis=1, inplace=True)
df_p.drop(name2, axis=1, inplace=True)
df_p.plot.area(figsize=(30, 10), grid=True, stacked=False, title="POINTS ADVANTAGE OVER ROUNDS")

In [ ]:
df['EARN_BY'][name1] = pd.Series(df['EARN_BY'][name1])
df['EARN_BY'][name2] = pd.Series(df['EARN_BY'][name2])
df['USED'][name1] = pd.Series(df['USED'][name1])
df['USED'][name2] = pd.Series(df['USED'][name2])

In [ ]:
methods = {'NOP()': 0, 'PATCH()': 0, 'SCAN()': 0, 'OVERLOAD()': 0, 'OVERHEAR()': 0, 'EXPLOIT()': 0, 'INFECT()': 0}
for b in df['USED']:
    for m in b:
        methods[m] += 1
        
pd.Series(methods).plot.pie(subplots=True, figsize=(10, 10), autopct='%.2f', title=["METHOD USED IN GAME"])

In [ ]:
dfe = pd.DataFrame({
    #f'{name1} - used': df['USED'][name1].value_counts(),
    f'{name1} - earn': df['EARN_BY'][name1],
    #f'{name2} - used': df['USED'][name2].value_counts(),
    f'{name2} - earn': df['EARN_BY'][name2],
})
dfe.plot.bar(figsize=(15, 5), grid=True, title="Points Earned by Methods")

In [ ]:
df['USED'][name1] = pd.Series(df['USED'][name1])
df['USED'][name2] = pd.Series(df['USED'][name2])

In [ ]:
dfu = pd.DataFrame({name1: df['USED'][name1].value_counts(),
                   name2: df['USED'][name2].value_counts()})

In [ ]:
dfu.plot.bar(figsize=(15, 5), grid=True, title="METHODS USED", color=['r', 'b'])

In [ ]:
dfu.plot.pie(subplots=True, figsize=(25, 25), autopct='%.2f', title=["METHOD USED", "METHOD USED"])

In [ ]:
df_win = pd.read_json(json.dumps(bot['WON_WITH']))

In [ ]:
df_win.plot.pie(subplots=True, figsize=(25, 25), autopct='%.2f', title=["WON BY METHOD", "WON BY METHOD"])

In [ ]:
df_lost = pd.read_json(json.dumps(bot['LOST_WITH']))

In [ ]:
df_lost.plot.pie(subplots=True, figsize=(25, 25), autopct='%.2f', title=["LOST BY METHOD", "LOST BY METHOD"])

In [ ]:
df_win.plot.bar(figsize=(15, 5), grid=True, title="WON WITH METHOD", color=['r', 'b'])

In [ ]:
df_lost.plot.bar(figsize=(15, 5), grid=True, title='LOST WITH METHOD', color=['r', 'b'])

In [ ]:
win = df_win[name1].to_dict()
lost = df_lost[name1].to_dict()

df1 = pd.DataFrame({'Lost': lost, 'Won': win})
df1.plot.bar(stacked=True, grid=True, figsize=(15, 5), title=f'{name1} - Won/Lost by Method')

In [ ]:
win = df_win[name2].to_dict()
lost = df_lost[name2].to_dict()

df1 = pd.DataFrame({'Lost': lost, 'Won': win})
df1.plot.bar(stacked=True, grid=True, figsize=(15, 5), title=f'{name2} - Won/Lost by Method')